In [ ]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import ipywidgets
from ipywidgets import interact

# Grover algorithm walktrough

## Initilize to $\ket{0^n}$ and apply Hadamards $\ket{s} = H^{\otimes n}\ket{0^n}$

In [ ]:
# Parameters to change
n = 3 # num qubits
N = 2**n # num possible states

omega = np.random.randint(N) # which index is phase-flipped

start_state = qt.ket("0"*n).unit() # initialize |0^n> = |000....0>
Hn = qt.gates.hadamard_transform(n)
s = qt.gates.hadamard_transform(n)*start_state # apply H on all |0>'s
if n < 3: # avoid printing too long state vectors
    print(start_state.basis_expansion())
s

## Create oracle $U_\omega$


The oprator must satisfy:
$$
    U_\omega\ket{x} = (-1)^{f(x)}\ket{x}\equiv\left\{ \begin{array}{lc} 
        U_\omega \ket{x}  = -\ket{x}, & x=\omega,\ f(x) = 1\\
        U_\omega \ket{x} = \ket{x}, & x\neq \omega, f(x) = 0
    \end{array}\right.
$$

In [ ]:
I = qt.qeye_like(s.proj())
w = qt.ket(np.binary_repr(omega, width=n))
oracle = I - 2*w.proj()
oracle

## Create Grover diffusion operator, $U_s$
$$
    U_s = H^{\otimes n } \Big(2\ket{0^n}\bra{0^n} - I_n\Big)H^{\otimes n } = 2\ket{s}\bra{s} - I_n
$$

<img src="Grover_circuit.jpg">


In [ ]:
Us = 2*s.proj() - qt.qeye_like(s.proj()) # 2|s><s| - I
Us

## Grover iterations

In [ ]:
rN = int(np.ceil(np.pi/4*np.sqrt(N)))
print(f"{rN = }")
out = s
iter = 0
path = [out]
for i in range(rN):
    # print(f"Iter : {iter+1}/{rN}")
    out = oracle * out
    out = Us * out
    path.append(out)
    # iter += 1

In [ ]:
iter = 0
one, zero = qt.ket("1"), qt.ket("0")
basis_bits = [f"{np.binary_repr(i, width=n)}" for i in range(len(s.full()))]
# print(basis_bits)

basis_states = [qt.ket(basis) for basis in basis_bits]
probs = np.abs(path[iter].full().squeeze())**2



fig, ax = plt.subplots(figsize=(20,6))
ax.bar(range(len(basis_bits)), [0]*(omega) + [1] + [0]*(N-1 - omega), label=rf"$\omega = {omega}$", alpha=0.7)
ax.bar([rf"$\vert{i}\rangle$" for i in range(len(basis_bits))], probs, alpha=0.8, label="Grover's")
ax.set_ylabel(r"$P_\omega(\vert x\rangle)$", size=15)
ax.set_xlabel(xlabel="Basis states", size=15)
ax.set_title(f"Grover's results for $n$={n}, $r(N) = {rN}$", size=18)
ax.grid()
ax.legend()
None

# Plot functions

In [ ]:
def grovers(num_bits=3, omega=2):
    """Compute Grover' algorithm for a specified number of bits and a oracle function with omega

    Parameters
    ----------
    num_bits : int, optional
        Number of qubits used for calcualtions, by default 3
    omega : int or list[int,], optional
        The omega(s) that yields a phase of -1. Can be an integer or a list of integers, by default 2.

    Returns
    -------
    psi_evo : ndarray
        evolution of Hadamard transformed input state in standard basis, {|0...00>, |0...01>, |0...10>, ..., |1..11>}
    s_evo : ndarray
        evolution of Hadamard transformed input state in {|w>, |s>} basis
    sp_evo : ndarray
        evolution of Hadamard transformed input in {|w>, |s'>} basis (orthornormal)
    """
    if isinstance(omega, int):
        omega = np.array([omega])
    Nw = np.size(omega)
    assert np.all(omega < 2**num_bits), f"omega has to be in range [0, N-1], for N=2**num_bits. Omega was: {omega}"
    N = 2**num_bits # num possible states
    total_iter = 100
    
    rN = np.pi/4 * np.sqrt(N/np.size(omega)) - 1/2
    print(f"Expected number of iterations: {rN:.2f}")
    
    s_rot = np.array([[-1,           0],
                      [2*np.sqrt(Nw)/np.sqrt(N), 1]])
    
    w_rot = np.array([[-1, -2*np.sqrt(Nw)/np.sqrt(N)],
                      [0,           1]])
    M = s_rot@w_rot # U_s * U_w
    
    s_evo = np.zeros(shape=(total_iter+1, 2)) # each row is the new 
    s_evo[0, :] = np.array([0, 1]) # save to array in basis {|w>, |s>}
    for iter in range(total_iter): # include 100 iterations to show evolution long after expected upper limit.
        # compute |s> in basis { |s'>, |w> }
        s_evo[iter+1, :] = M@s_evo[iter,:]
        
    basis_change = np.array([[1, np.sqrt(Nw)/np.sqrt(N)],
                             [0, np.sqrt((N-Nw)/N)]])
    
    kets = np.zeros((2,N))
    for i in omega: # |w>
        kets[0,i] = 1/np.sqrt(Nw)
    kets[1,:] = 1/np.sqrt(N) # |s>
    psi_evo = s_evo @ kets # compute evolution in standard basis

    sp_evo = (basis_change@s_evo[:, :, None]).squeeze() # {|w>, |s>} -> {|w>, |s'>}

    return psi_evo, s_evo, sp_evo




In [ ]:
def plot_vector(sp_evo, iter, omega, ax=None):
    """Plot the 2D repr of the Grover's algorithm iterations

    Parameters
    ----------
    sp_evo : numpy array
         array containing the |s'> repr of the Grover's system iterations
    iter : int
        Which iteration of sp_evo to investigate
    ax : matplotlib.Axes, optional
        The axis for plot used if combined with other plots, by default None
    """
    if isinstance(omega, int):
        omega = np.array([omega])
    if ax is None:
        fig, ax = plt.subplots(1,1, figsize=(6,6))
        if len(omega) < 5:
            ax.set_title(f"Grover's algorithm\n $n={n}$ qubits, $\\omega = {tuple(int(i) for i in omega)}$", size=16)
        else:
            ax.set_title(f"Grover's algorithm\n $n={n}$ qubits, $N_\\omega = {len(omega)}$ omega states", size=16)
        
    ax.quiver(0, 0, 0, 1, angles='xy', scale_units='xy', scale=1, color="b")
    ax.annotate(r"$\vert {\omega}\rangle$", xytext=(-0.15, 1-0.05), xy=(0,0), color="b")
    ax.quiver(0, 0, 1, 0, angles='xy', scale_units='xy', scale=1, color="r")
    ax.annotate(r"$\vert {s'}\rangle$", xytext=(1-0.05, +0.05), xy=(0,0), color="r")
    
    # plot starting point
    y_start, x_start = sp_evo[0] # saved in the opposite order (y,x) but we want to plot as (x,y)
    ax.annotate("", xytext=(0,0), xy=(x_start, y_start), arrowprops=dict(arrowstyle="->", linestyle="--", alpha=0.7))
    ax.quiver(0, 0, x_start, y_start, angles="xy", scale_units="xy", scale=1, color="k", alpha=0.7)
    ax.annotate(r"$\vert {s}\rangle$", xytext=(x_start+0.05*x_start, y_start+0.05*y_start), xy=(0,0)) 
    if iter != 0:
        y,x = sp_evo[iter] # saved in the opposite order (y,x) but we want to plot as (x,y)
        # ax.annotate("", xytext=(0,0), xy=(x, y), arrowprops=dict(arrowstyle="->", linestyle="--", alpha=0.7))
        ax.quiver(0, 0, x, y, angles="xy", scale_units="xy", scale=1, color="k", alpha=0.7)
        ax.annotate(f"Iter {iter}", xytext=(x+0.05*x, y+0.05*y), xy=(0,0))
    
    # plot half unit circle 
    thetas = np.linspace(0, np.pi, num=100)
    circ_x, circ_y = np.cos(thetas), np.sin(thetas)
    ax.plot(circ_x, circ_y, "--k")
    
    ax.set(xlim=(-1.1,1.1), ylim=(-0.1,1.1))
    ax.set_xticks(np.linspace(-1, 1, num=9, endpoint=True))
    ax.set_yticks(np.linspace(0, 1, num=5, endpoint=True))
    ax.set_aspect("equal"   , adjustable="box")
    ax.grid()
    
def plot_bar(psi_evo, iter, omega, ax=None):
    """Barplot of all states of Grover's algorithm iterations

    Parameters
    ----------
    psi_evo : list[qt.Qobj]
         list of qutip object for the states in each iteration of Grover's algorithm
    iter : int
        Which iteration of psi_evo to investigate
    ax : matplotlib.Axes, optional
        The axis for plot used if combined with other plots, by default None
    """
    if isinstance(omega, int):
        omega = np.array([omega])
    
    Nw = np.size(omega)
    if ax is None:
        fig, ax = plt.subplots(1,1, sharey=True)
        if len(omega) < 5:
            fig.suptitle(f"Grover's algorithm\n $n={n}$ qubits, $\\omega = {tuple(int(i) for i in omega)}$", size=16)
        else:
            fig.suptitle(f"Grover's algorithm\n $n={n}$ qubits, $N_\\omega = {len(omega)}$ omega states", size=16)
        
    
    N = psi_evo.shape[1]
    probs = np.abs(psi_evo[iter])**2
    
    w_arr_probs = np.zeros(shape=(N))
    w_arr_probs[omega] = 1/Nw
    
    ax.bar(range(N), w_arr_probs, label=rf"$\omega = {tuple(int(i) for i in omega)}$", alpha=0.8,)
    ax.bar([rf"$\vert{i}\rangle$" for i in range(N)], probs, label="Grover's")
    ax.set_ylim([0, np.max(w_arr_probs)*1.3])
    ax.legend()
    

def plot_combined(psi_evo, sp_evo, omega, iter):
    """Plot the 2D repr of the Grover's algorithm iterations and the states in the same plot

    Parameters
    ----------
    psi_evo : list[qt.Qobj]
         list of qutip object for the states in each iteration of Grover's algorithm
    sp_evo : numpy array
         array containing the |s'> repr of the Grover's system iterations
    iter : int
        Which iteratio of sp_evo to investigate
    ax : matplotlib.Axes, optional
        The axis for plot used if combined with other plots, by default None
    """
    if isinstance(omega, int):
        omega = np.array([omega])
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # plot |s> in  {|s'>, |w>} basis
    plot_vector(sp_evo, iter, omega, ax=axes[0])

    
    # make barplot of basis states in |s>
    plot_bar(psi_evo, iter, omega, ax=axes[1])
    
    pos0 = axes[0].get_position()
    pos1 = axes[1].get_position()
    
    axes[1].set_position([
        pos1.x0,        # keep same left x
        pos0.y0,        # match the lower y
        pos1.width*1.2,     # keep original width
        pos0.height     # match the height of ax0
    ])

    if len(omega) < 5:
        fig.suptitle(f"Grover's algorithm\n $n={n}$ qubits, $\\omega = {tuple(int(i) for i in omega)}$", size=18)
    else:
        fig.suptitle(f"Grover's algorithm\n $n={n}$ qubits, $N_\\omega = {len(omega)}$ omega states", size=18)

## Interactive of vector and bar plots

In [ ]:
n = 4# num qubits   
N = 2**n # dimension of Hilbert space
omega = np.random.choice(range(N), size=2, replace=False) # a random omega in range [0, N-1] 



psi_evo, s_evo, sp_evo = grovers(n, omega)
iter_slider = ipywidgets.IntSlider(value=0, min=0, max=len(psi_evo)-1)
psi_static = ipywidgets.fixed(psi_evo)
sp_static = ipywidgets.fixed(sp_evo)
omega_static = ipywidgets.fixed(omega)

interact(plot_combined, iter=iter_slider, sp_evo=sp_static, psi_evo=psi_static, omega=omega_static)
None


## Animation of vector and bar plots

In [ ]:
n = 4
N = 2**n
omega = np.random.choice(range(N), replace=False, size=1)

psi_evo, _, sp_evo = grovers(n, omega)

n_frames = len(sp_evo)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
if np.size(omega) < 3:
    fig.suptitle(f"Grover's algorithm, $n={n}$ qubits, $\\omega = {tuple(int(i) for i in omega)}$", size=16)
else:
    fig.suptitle(f"Grover's algorithm, $n={n}$ qubits, $N_\\omega = {np.size(omega)}$", size=16)

def update(frame):
    axes[0].clear()
    axes[1].clear()

    # Update plots for current frame
    plot_vector(sp_evo, frame, omega, ax=axes[0])
    plot_bar(psi_evo, frame, omega, ax=axes[1])

    # Optionally adjust layout if needed
    pos0 = axes[0].get_position()
    pos1 = axes[1].get_position()
    axes[1].set_position([pos1.x0, pos0.y0, pos1.width, pos0.height])


anim = FuncAnimation(fig, update, frames=n_frames, interval=500); 
plt.close(fig)
HTML(anim.to_jshtml())


## Interactive for vector plot

In [ ]:
n = 10
N = 2**n
omega = np.random.randint(N, size=4)
*_, sp = grovers(n, omega)


sp_static = ipywidgets.fixed(sp)
iter_slider = ipywidgets.IntSlider(value=0, min=0, max=len(sp)-1, indent=True)
ax_static = ipywidgets.fixed(None)
omega_static = ipywidgets.fixed(omega)
interact(plot_vector, sp_evo=sp_static, iter=iter_slider, ax=ax_static, omega=omega_static)

## Animation for vector plot

In [ ]:
n = 10
N = 2**n
omega = np.random.choice(range(N), replace=False, size=10)

psi_evo, _, sp_evo = grovers(n, omega)

n_frames = len(sp_evo)

fig, axes = plt.subplots(1, 1, figsize=(12, 4))
if np.size(omega) < 3:
    fig.suptitle(f"Grover's algorithm, $n={n}$ qubits, $\\omega = {tuple(int(i) for i in omega)}$", size=16)
else:
    fig.suptitle(f"Grover's algorithm, $n={n}$ qubits, $N_\\omega = {np.size(omega)}$", size=16)
    

def update(frame):
    axes.clear()

    # Update plots for current frame
    plot_vector(sp_evo, frame, omega, ax=axes)



anim = FuncAnimation(fig, update, frames=n_frames, interval=500); 
plt.close(fig)
HTML(anim.to_jshtml())
